In [1]:
# Install all required packages
!pip install gradio
!pip install openai-whisper
!pip install gTTS
!pip install twilio
!pip install pydub
!pip install python-dotenv
!pip install requests

# Import all libraries
import gradio as gr
import whisper
import os
import tempfile
import uuid
from datetime import datetime
import json
from gtts import gTTS
from twilio.rest import Client
import re
from pydub import AudioSegment
import io
import base64

print("✅ All packages installed successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.2 MB/

In [2]:
# Complete welfare schemes database
WELFARE_SCHEMES = {
    'pmay': {
        'name': 'Pradhan Mantri Awas Yojana (PM-AY)',
        'description': 'Housing scheme for economically weaker sections',
        'eligibility': 'Annual income below ₹3-6 lakh, no pucca house',
        'benefits': 'Subsidy up to ₹2.67 lakh for house construction',
        'keywords': ['house', 'housing', 'home', 'shelter', 'construction', 'गृह', 'मकान', 'आवास'],
        'categories': ['housing', 'urban', 'rural']
    },
    'ujjwala': {
        'name': 'PM Ujjwala Yojana',
        'description': 'Free LPG connection for women from BPL families',
        'eligibility': 'BPL family, woman as beneficiary',
        'benefits': 'Free LPG connection + ₹1600 deposit',
        'keywords': ['gas', 'lpg', 'cooking', 'fuel', 'cylinder', 'गैस', 'रसोई', 'खाना'],
        'categories': ['fuel', 'women', 'cooking']
    },
    'pmkisan': {
        'name': 'PM-KISAN Samman Nidhi',
        'description': 'Income support for small farmer families',
        'eligibility': 'Small & marginal farmers with cultivable land',
        'benefits': '₹6000 per year in 3 installments',
        'keywords': ['farmer', 'agriculture', 'farming', 'crop', 'land', 'किसान', 'खेती', 'फसल'],
        'categories': ['agriculture', 'farming', 'income']
    },
    'ayushman': {
        'name': 'Ayushman Bharat (PM-JAY)',
        'description': 'Health insurance for poor families',
        'eligibility': 'Families identified in SECC database',
        'benefits': 'Health cover up to ₹5 lakh per family per year',
        'keywords': ['health', 'medical', 'insurance', 'hospital', 'treatment', 'स्वास्थ्य', 'इलाज', 'बीमा'],
        'categories': ['health', 'insurance', 'medical']
    },
    'pension': {
        'name': 'National Social Assistance Programme (NSAP)',
        'description': 'Pension for elderly, widow, and disabled',
        'eligibility': 'Age 60+, widows, persons with disabilities',
        'benefits': '₹200-500 per month pension',
        'keywords': ['pension', 'elderly', 'old', 'widow', 'disabled', 'पेंशन', 'बुजुर्ग', 'विधवा'],
        'categories': ['pension', 'elderly', 'social_security']
    },
    'scholarship': {
        'name': 'Post Matric Scholarship',
        'description': 'Educational support for SC/ST students',
        'eligibility': 'SC/ST students in post-matric courses',
        'benefits': 'Tuition fees + maintenance allowance',
        'keywords': ['education', 'study', 'student', 'scholarship', 'school', 'college', 'शिक्षा', 'पढ़ाई', 'छात्रवृत्ति'],
        'categories': ['education', 'student', 'scholarship']
    },
    'mudra': {
        'name': 'Pradhan Mantri MUDRA Yojana',
        'description': 'Micro finance loans for small businesses',
        'eligibility': 'Micro enterprises, small traders, self-employed',
        'benefits': 'Loans up to ₹10 lakh without collateral',
        'keywords': ['business', 'loan', 'money', 'enterprise', 'trade', 'व्यापार', 'कारोबार', 'ऋण'],
        'categories': ['business', 'loan', 'finance']
    },
    'swachh': {
        'name': 'Swachh Bharat Mission',
        'description': 'Toilet construction in rural areas',
        'eligibility': 'Households without toilets in rural areas',
        'benefits': '₹12,000 incentive for toilet construction',
        'keywords': ['toilet', 'sanitation', 'cleanliness', 'bathroom', 'शौचालय', 'स्वच्छता', 'सफाई'],
        'categories': ['sanitation', 'rural', 'infrastructure']
    }
}

print("✅ Welfare schemes database loaded!")
print(f"📊 Total schemes available: {len(WELFARE_SCHEMES)}")


✅ Welfare schemes database loaded!
📊 Total schemes available: 8


In [3]:
# Load Whisper model for speech recognition
print("🎤 Loading Whisper model...")
whisper_model = whisper.load_model("base")
print("✅ Whisper model loaded!")

def transcribe_audio(audio_file_path, language="auto"):
    """Convert speech to text using Whisper"""
    try:
        # Load and transcribe audio
        result = whisper_model.transcribe(audio_file_path)

        return {
            'success': True,
            'text': result['text'].strip(),
            'language': result.get('language', 'unknown')
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'text': '',
            'language': 'unknown'
        }

def extract_keywords_and_intent(text):
    """Extract relevant information from user text"""
    text = text.lower()

    # Extract potential income
    income_match = re.search(r'(\d+)\s*(lakh|thousand|हजार|लाख)', text)
    income = None
    if income_match:
        amount = int(income_match.group(1))
        unit = income_match.group(2)
        if 'lakh' in unit or 'लाख' in unit:
            income = amount * 100000
        else:
            income = amount * 1000

    # Extract age
    age_match = re.search(r'(\d+)\s*(year|साल|वर्ष)', text)
    age = int(age_match.group(1)) if age_match else None

    # Extract location type
    location_type = 'rural' if any(word in text for word in ['village', 'rural', 'गांव', 'ग्रामीण']) else 'urban'

    # Extract gender
    gender = None
    if any(word in text for word in ['woman', 'female', 'wife', 'mother', 'महिला', 'स्त्री']):
        gender = 'female'
    elif any(word in text for word in ['man', 'male', 'husband', 'father', 'पुरुष', 'आदमी']):
        gender = 'male'

    return {
        'text': text,
        'income': income,
        'age': age,
        'location_type': location_type,
        'gender': gender,
        'keywords': text.split()
    }

def match_welfare_schemes(user_info):
    """Find matching welfare schemes based on user information"""
    matched_schemes = []

    for scheme_id, scheme in WELFARE_SCHEMES.items():
        score = 0

        # Keyword matching
        user_keywords = user_info['keywords']
        scheme_keywords = scheme['keywords']

        keyword_matches = sum(1 for keyword in scheme_keywords
                            if any(uk in keyword or keyword in uk for uk in user_keywords))
        score += keyword_matches * 10

        # Income-based matching
        if user_info['income']:
            if scheme_id == 'pmay' and user_info['income'] < 600000:
                score += 20
            elif scheme_id == 'pmkisan' and user_info['income'] < 200000:
                score += 20
            elif scheme_id == 'ujjwala' and user_info['income'] < 100000:
                score += 20

        # Age-based matching
        if user_info['age']:
            if scheme_id == 'pension' and user_info['age'] >= 60:
                score += 30
            elif scheme_id == 'scholarship' and 18 <= user_info['age'] <= 25:
                score += 25

        # Gender-based matching
        if user_info['gender'] == 'female' and scheme_id == 'ujjwala':
            score += 15

        if score > 10:  # Minimum threshold
            matched_schemes.append({
                'id': scheme_id,
                'score': score,
                **scheme
            })

    # Sort by score and return top matches
    matched_schemes.sort(key=lambda x: x['score'], reverse=True)
    return matched_schemes[:3]  # Return top 3 matches

def generate_response_text(schemes, language="hi"):
    """Generate appropriate response text"""
    if not schemes:
        if language == "hi":
            return "खुशी की बात है कि आपने संपर्क किया। अभी आपके लिए कोई विशेष योजना नहीं मिली, लेकिन कृपया अपनी जरूरतों के बारे में और बताएं।"
        else:
            return "Thank you for your inquiry. I couldn't find specific schemes matching your needs right now. Please provide more details about your requirements."

    if language == "hi":
        if len(schemes) == 1:
            return f"आपके लिए एक योजना मिली है: {schemes[0]['name']}। {schemes[0]['description']}। अधिक जानकारी नीचे दी गई है।"
        else:
            scheme_names = ", ".join([s['name'] for s in schemes])
            return f"आपके लिए {len(schemes)} योजनाएं मिली हैं: {scheme_names}। विस्तृत जानकारी नीचे दी गई है।"
    else:
        if len(schemes) == 1:
            return f"I found one scheme for you: {schemes[0]['name']}. {schemes[0]['description']}. Details are provided below."
        else:
            scheme_names = ", ".join([s['name'] for s in schemes])
            return f"I found {len(schemes)} schemes for you: {scheme_names}. Detailed information is provided below."

def text_to_speech(text, language="hi"):
    """Convert text to speech using gTTS"""
    try:
        # Map language codes
        lang_code = "hi" if language == "hi" else "en"

        # Generate TTS
        tts = gTTS(text=text, lang=lang_code, slow=False)

        # Save to temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tts.save(temp_file.name)

        return {
            'success': True,
            'file_path': temp_file.name
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'file_path': None
        }

print("✅ AI processing functions ready!")


🎤 Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 107MiB/s]


✅ Whisper model loaded!
✅ AI processing functions ready!


In [4]:
# Twilio SMS setup (optional - set your credentials)
TWILIO_ACCOUNT_SID = ""  # Add your Twilio Account SID
TWILIO_AUTH_TOKEN = ""   # Add your Twilio Auth Token
TWILIO_PHONE_NUMBER = "" # Add your Twilio phone number

def send_sms(phone_number, message):
    """Send SMS via Twilio (if credentials are provided)"""
    if not all([TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER]):
        return {
            'success': False,
            'error': 'SMS credentials not configured',
            'message': 'SMS functionality disabled - no Twilio credentials'
        }

    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        message = client.messages.create(
            body=message,
            from_=TWILIO_PHONE_NUMBER,
            to=phone_number
        )
        return {
            'success': True,
            'message_sid': message.sid,
            'message': 'SMS sent successfully'
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'message': 'Failed to send SMS'
        }

def generate_sms_text(schemes, language="hi"):
    """Generate SMS text with scheme details"""
    if language == "hi":
        sms = "🏛️ आपकी योजनाएं:\n\n"
        for i, scheme in enumerate(schemes, 1):
            sms += f"{i}. {scheme['name']}\n"
            sms += f"   📋 पात्रता: {scheme['eligibility']}\n"
            sms += f"   💰 लाभ: {scheme['benefits']}\n\n"
        sms += "अधिक जानकारी के लिए नजदीकी सरकारी कार्यालय में संपर्क करें।"
    else:
        sms = "🏛️ Your Eligible Schemes:\n\n"
        for i, scheme in enumerate(schemes, 1):
            sms += f"{i}. {scheme['name']}\n"
            sms += f"   📋 Eligibility: {scheme['eligibility']}\n"
            sms += f"   💰 Benefits: {scheme['benefits']}\n\n"
        sms += "For more information, contact your nearest government office."

    return sms

print("✅ SMS integration ready! (Configure Twilio credentials to enable)")


✅ SMS integration ready! (Configure Twilio credentials to enable)


In [5]:
def process_voice_query(audio_file, phone_number="", language_preference="auto"):
    """Main function to process voice query and return welfare scheme information"""

    session_id = str(uuid.uuid4())[:8]
    processing_log = []

    try:
        # Step 1: Speech to Text
        processing_log.append("🎤 Converting speech to text...")

        if audio_file is None:
            return "❌ Please upload an audio file", "", [], None, "No audio file provided"

        transcription_result = transcribe_audio(audio_file)

        if not transcription_result['success']:
            return f"❌ Speech recognition failed: {transcription_result['error']}", "", [], None, "\n".join(processing_log)

        transcribed_text = transcription_result['text']
        detected_language = transcription_result['language']

        processing_log.append(f"✅ Transcribed: '{transcribed_text}'")
        processing_log.append(f"🌐 Language detected: {detected_language}")

        # Step 2: Extract information
        processing_log.append("🧠 Analyzing your requirements...")
        user_info = extract_keywords_and_intent(transcribed_text)

        # Step 3: Match schemes
        processing_log.append("🔍 Finding matching welfare schemes...")
        matching_schemes = match_welfare_schemes(user_info)

        processing_log.append(f"✅ Found {len(matching_schemes)} matching schemes")

        # Step 4: Generate response
        response_text = generate_response_text(matching_schemes, detected_language)

        # Step 5: Generate TTS
        processing_log.append("🔊 Generating voice response...")
        tts_result = text_to_speech(response_text, detected_language)

        tts_audio_file = None
        if tts_result['success']:
            tts_audio_file = tts_result['file_path']
            processing_log.append("✅ Voice response generated")
        else:
            processing_log.append(f"⚠️ TTS failed: {tts_result['error']}")

        # Step 6: Send SMS (if phone number provided)
        sms_status = ""
        if phone_number.strip():
            processing_log.append("📱 Sending SMS with detailed information...")
            sms_text = generate_sms_text(matching_schemes, detected_language)
            sms_result = send_sms(phone_number, sms_text)
            if sms_result['success']:
                sms_status = "✅ SMS sent successfully"
            else:
                sms_status = f"⚠️ SMS failed: {sms_result['message']}"
            processing_log.append(sms_status)

        # Format schemes for display
        schemes_display = []
        for scheme in matching_schemes:
            schemes_display.append({
                "name": scheme['name'],
                "description": scheme['description'],
                "eligibility": scheme['eligibility'],
                "benefits": scheme['benefits'],
                "relevance_score": f"{scheme['score']}/100"
            })

        return (
            response_text,
            transcribed_text,
            schemes_display,
            tts_audio_file,
            "\n".join(processing_log) + f"\n\n{sms_status}"
        )

    except Exception as e:
        error_msg = f"❌ Error processing request: {str(e)}"
        processing_log.append(error_msg)
        return error_msg, "", [], None, "\n".join(processing_log)

def process_text_query(text_input, phone_number="", language_preference="hi"):
    """Process text query directly (for testing without audio)"""

    session_id = str(uuid.uuid4())[:8]
    processing_log = []

    try:
        if not text_input.strip():
            return "❌ Please enter your query", "", [], None, "No text input provided"

        processing_log.append(f"📝 Processing text: '{text_input}'")

        # Extract information
        user_info = extract_keywords_and_intent(text_input)
        processing_log.append("🧠 Analyzing your requirements...")

        # Match schemes
        matching_schemes = match_welfare_schemes(user_info)
        processing_log.append(f"✅ Found {len(matching_schemes)} matching schemes")

        # Generate response
        response_text = generate_response_text(matching_schemes, language_preference)

        # Generate TTS
        processing_log.append("🔊 Generating voice response...")
        tts_result = text_to_speech(response_text, language_preference)

        tts_audio_file = None
        if tts_result['success']:
            tts_audio_file = tts_result['file_path']
            processing_log.append("✅ Voice response generated")

        # Send SMS (if phone number provided)
        sms_status = ""
        if phone_number.strip():
            processing_log.append("📱 Sending SMS...")
            sms_text = generate_sms_text(matching_schemes, language_preference)
            sms_result = send_sms(phone_number, sms_text)
            sms_status = "✅ SMS sent" if sms_result['success'] else f"⚠️ SMS failed: {sms_result['message']}"
            processing_log.append(sms_status)

        # Format schemes for display
        schemes_display = []
        for scheme in matching_schemes:
            schemes_display.append({
                "name": scheme['name'],
                "description": scheme['description'],
                "eligibility": scheme['eligibility'],
                "benefits": scheme['benefits'],
                "relevance_score": f"{scheme['score']}/100"
            })

        return (
            response_text,
            text_input,
            schemes_display,
            tts_audio_file,
            "\n".join(processing_log) + f"\n\n{sms_status}"
        )

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        processing_log.append(error_msg)
        return error_msg, "", [], None, "\n".join(processing_log)

print("✅ Main processing functions ready!")


✅ Main processing functions ready!


In [7]:
# Create the Gradio web interface
def create_interface():

    with gr.Blocks(title="Unlocking Access - Voice-First Welfare Assistant", theme=gr.themes.Soft()) as demo:

        gr.Markdown("""
        # Unlocking Access - Voice-First AI Platform for Social Welfare
        ### Hack-A-Tone 2025 MVP Demo

        **How it works:**
        1.  Upload audio file OR type your query
        2.  AI processes and matches welfare schemes
        3.  Get voice response + detailed information
        4.  Optional SMS with complete details

        **Languages Supported:** Hindi, English, and mixed (Hinglish)
        """)

        with gr.Tab(" Voice Query"):
            with gr.Row():
                with gr.Column():
                    audio_input = gr.Audio(
                        sources=["microphone", "upload"],
                        type="filepath",
                        label="Record or Upload Audio Query"
                    )
                    phone_input_voice = gr.Textbox(
                        label=" Phone Number (Optional)",
                        placeholder="+91 1234567891",
                        info="Enter phone number to receive SMS with detailed scheme information"
                    )
                    lang_voice = gr.Dropdown(
                        choices=["auto", "hi", "en"],
                        value="auto",
                        label="Language Preference"
                    )
                    process_voice_btn = gr.Button("Process Voice Query", variant="primary")

                with gr.Column():
                    response_voice = gr.Textbox(label=" AI Response", lines=4)
                    transcription_voice = gr.Textbox(label="What You Said", lines=2)
                    audio_response_voice = gr.Audio(label=" Voice Response")

        with gr.Tab(" Text Query"):
            with gr.Row():
                with gr.Column():
                    text_input = gr.Textbox(
                        label="Type Your Query",
                        lines=3,
                        placeholder="Example: मुझे घर बनाने के लिए सरकारी योजना चाहिए (I need government scheme for house construction)"
                    )
                    phone_input_text = gr.Textbox(
                        label=" Phone Number (Optional)",
                        placeholder="+919876543210"
                    )
                    lang_text = gr.Dropdown(
                        choices=["hi", "en"],
                        value="hi",
                        label=" Response Language"
                    )
                    process_text_btn = gr.Button(" Process Text Query", variant="primary")

                with gr.Column():
                    response_text = gr.Textbox(label=" AI Response", lines=4)
                    echo_text = gr.Textbox(label=" Your Query", lines=2)
                    audio_response_text = gr.Audio(label=" Voice Response")

        # Shared outputs
        with gr.Row():
            schemes_output = gr.JSON(label="Matched Welfare Schemes", visible=True)

        with gr.Row():
            processing_log = gr.Textbox(label=" Processing Log", lines=8, max_lines=10)

        # Event handlers
        process_voice_btn.click(
            fn=process_voice_query,
            inputs=[audio_input, phone_input_voice, lang_voice],
            outputs=[response_voice, transcription_voice, schemes_output, audio_response_voice, processing_log]
        )

        process_text_btn.click(
            fn=process_text_query,
            inputs=[text_input, phone_input_text, lang_text],
            outputs=[response_text, echo_text, schemes_output, audio_response_text, processing_log]
        )

        # Example queries
        gr.Markdown("""
        ### Sample Queries to Try:

        **Hindi Examples:**
        - मुझे घर बनाने के लिए योजना चाहिए (I need scheme for house construction)
        - किसान के लिए क्या योजना है (What schemes for farmers)
        - बुजुर्गों के लिए पेंशन की जानकारी (Pension information for elderly)
        - महिला के लिए गैस कनेक्शन (Gas connection for women)

        **English Examples:**
        - I need financial help for my small business
        - What health insurance schemes are available
        - Education scholarship for my daughter
        - I am 65 years old and need pension information
        """)

    return demo

# Launch the interface
demo = create_interface()
print("Starting Gradio interface...")
demo.launch(share=True, debug=True)


Starting Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2843d4ad9835ff6cb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2843d4ad9835ff6cb0.gradio.live
